# Assignment

In this assignment we will train a multilayer perceptron model on the MNIST dataset, extending the linear (single-layer) classifier in the tutorial. 

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook (push to https://github.com/[username]/cs190/mlp/assignment.ipynb)
* final trained `*.hdf5` model file

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment. For those interested in running a local Jupyter server, please consider using one of the following options to ensure dependency compatibility:

1. Precompiled Docker images: see https://github.com/peterchang77/install (**recommended**)
2. Conda environment files: see https://github.com/peterchang77/dl_utils/tree/master/envs

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [0]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [0]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [0]:
# --- Select Tensorflow 2.0 (only in Google Colab)
%tensorflow_version 2.x

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 276kB 8.3MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=3fffae9d5eddd64538df8d202ef0fab4fbe60a7b0d4cc1994e2db46f97ae6324
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [0]:
import numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from jarvis.train import datasets

# Data

As in the tutorial, data for this assignment will consist of the MNIST handwritten digit dataset. The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset
3. Prepare the corresponding Tensorflow Input(...) objects for model definition

In [3]:
# --- Download dataset
datasets.download(name='mnist')

# --- Prepare generators and model inputs
gen_train, _, client = datasets.prepare(name='mnist')
inputs = client.get_inputs(Input)

[ 2020-05-31 06:18:19 ] [====================] 100.000% : Iterating | 000001    

**Note**: There is no need to change the above code for this assignment.

# Training

In this assignment we will train a multilayer perceptron, e.g. a simple neural network with at least one hidden layer. Be creative; feel free to try various permutations of: 

* number(s) of hidden layer(s)
* size of hidden layer(s)
* learning rate
* training iterations 

### Define the model

In [0]:
# --- Define model
h1 = layers.Dense(128, activation = 'relu')(inputs['dat'])
h2 = layers.Dense(64, activation = 'relu')(h1)
h3 = layers.Dense(32, activation = 'relu')(h2)

logits = {}
logits['digit'] = layers.Dense(10, name='digit')(h3)
model = Model(inputs=inputs, outputs=logits)

### Compile the model

In [0]:
# --- Define loss and optimizer
loss = losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = optimizers.Adam(learning_rate=1e-2)

# --- Compile model
model.compile(
    optimizer = optimizer,
    loss = {'digit' : loss},
    metrics = {'digit' : 'sparse_categorical_accuracy'}
)

### Train the model

In [6]:
model.fit_generator(
    generator=gen_train, 
    steps_per_epoch=250, 
    epochs=4)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/4
250/250 [==============================] - 26s 104ms/step - loss: 0.9547 - sparse_categorical_accuracy: 0.7235
Epoch 2/4
250/250 [==============================] - 26s 106ms/step - loss: 0.3895 - sparse_categorical_accuracy: 0.8902
Epoch 3/4
250/250 [==============================] - 26s 106ms/step - loss: 0.2939 - sparse_categorical_accuracy: 0.9169
Epoch 4/4
250/250 [==============================] - 26s 104ms/step - loss: 0.2669 - sparse_categorical_accuracy: 0.9233


# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `model.predict(...)` on the training data. 

In [8]:
# --- Calculate accuracy
arrs = client.get(rows=np.arange(60000))
pred = model.predict(arrs['xs']['dat'])

# --- Serialize as *.csv file
df = pd.DataFrame(index=client.db.fnames.index)
df['true'] = arrs['ys']['digit'][:, 0]
df['pred'] = np.argmax(pred['digit'], axis = 1)
df['corr'] = df['true'] == df['pred']

# --- Print cumulative model perforance
df['corr'].mean()

0.914

**Note**: this cell is used only to check for model performance. It will not be graded. Once you are satisfied with your model, proceed to submission of your assignment below.

# Submission

Use the following line to save your model for submission (in Google Colab this should save your model file into your personal Google Drive):

In [0]:
# --- Serialize a model
fname = '{}/models/linear/final.hdf5'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
model.save(fname)

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooked file would be submitted under the name `panteater_notebook.ipynb` and his model file would be submitted under the name `panteater_model.hdf5`.